## Feature Overview

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>LoanStatNew</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>loan_amnt</td>
      <td>The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.</td>
    </tr>
    <tr>
      <th>2</th>
      <td>funded_amnt</td>
      <td>The total amount committed to that loan at that point in time.</td>
    </tr>
    <tr>
      <th>3</th>
      <td>funded_amnt_inv</td>
      <td>The total amount committed by investors for that loan at that point in time.</td>
    </tr>
    <tr>
      <th>4</th>
      <td>term</td>
      <td>The number of payments on the loan. Values are in months and can be either 36 or 60.</td>
    </tr>
    <tr>
      <th>5</th>
      <td>int_rate</td>
      <td>Interest Rate on the loan.</td>
    </tr>
    <tr>
      <th>6</th>
      <td>installment</td>
      <td>The monthly payment owed by the borrower if the loan originates.</td>
    </tr>
    <tr>
      <th>7</th>
      <td>grade</td>
      <td>LC assigned loan grade.</td>
    </tr>
    <tr>
      <th>8</th>
      <td>sub_grade</td>
      <td>LC assigned loan subgrade.</td>
    </tr>
    <tr>
      <th>9</th>
      <td>emp_title</td>
      <td>The job title supplied by the Borrower when applying for the loan.</td>
    </tr>
    <tr>
      <th>10</th>
      <td>emp_length</td>
      <td>Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.</td>
    </tr>
    <tr>
      <th>11</th>
      <td>home_ownership</td>
      <td>The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.</td>
    </tr>
    <tr>
      <th>12</th>
      <td>annual_inc</td>
      <td>The self-reported annual income provided by the borrower during registration.</td>
    </tr>
    <tr>
      <th>13</th>
      <td>verification_status</td>
      <td>Indicates if income was verified by LC, not verified, or if the income source was verified.</td>
    </tr>
    <tr>
      <th>14</th>
      <td>issue_d</td>
      <td>The month which the loan was funded.</td>
    </tr>
    <tr>
      <th>15</th>
      <td>loan_status</td>
      <td>Current status of the loan.</td>
    </tr>
    <tr>
      <th>16</th>
      <td>purpose</td>
      <td>A category provided by the borrower for the loan request.</td>
    </tr>
    <tr>
      <th>17</th>
      <td>addr_state</td>
      <td>The state provided by the borrower in the loan application.</td>
    </tr>
    <tr>
      <th>18</th>
      <td>dti</td>
      <td>A ratio calculated using the borrower's total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower's self-reported monthly income.</td>
    </tr>
    <tr>
      <th>19</th>
      <td>delinq_2yrs</td>
      <td>The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years.</td>
    </tr>
    <tr>
      <th>20</th>
      <td>earliest_cr_line</td>
      <td>The month the borrower's earliest reported credit line was opened.</td>
    </tr>
    <tr>
      <th>21</th>
      <td>fico_range_low</td>
      <td>The lower boundary range the borrower's FICO at loan origination belongs to.</td>
    </tr>
    <tr>
      <th>22</th>
      <td>fico_range_high</td>
      <td>The upper boundary range the borrower's FICO at loan origination belongs to.</td>
    </tr>
    <tr>
      <th>23</th>
      <td>inq_last_6mths</td>
      <td>The number of inquiries in the past 6 months (excluding auto and mortgage inquiries).</td>
    </tr>
    <tr>
      <th>24</th>
      <td>open_acc</td>
      <td>The number of open credit lines in the borrower's credit file.</td>
    </tr>
    <tr>
      <th>25</th>
      <td>pub_rec</td>
      <td>Number of derogatory public records.</td>
    </tr>
    <tr>
      <th>26</th>
      <td>revol_bal</td>
      <td>Total credit revolving balance.</td>
    </tr>
    <tr>
      <th>27</th>
      <td>revol_util</td>
      <td>Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.</td>
    </tr>
    <tr>
      <th>28</th>
      <td>total_acc</td>
      <td>The total number of credit lines currently in the borrower's credit file.</td>
    </tr>
  </tbody>
</table>

## Loading and Setup

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
# Filter warnings
from warnings import filterwarnings
filterwarnings('ignore')

/Users/rachelliao/anaconda3/envs/ensemble/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Load the dataset
df = pd.read_parquet('dataset/LoanData_after_EDA.parquet')

## Assessment

In [3]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# Print the shape of the data
df.shape

(20834, 28)

In [5]:
# Print the first 5 rows of the dataset
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,MORTGAGE,55000.0,Not Verified,Paid,debt_consolidation,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,7.0,0.0,2765.0,29.7,13.0,w,Individual
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,MORTGAGE,65000.0,Not Verified,Paid,small_business,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,22.0,0.0,21470.0,19.2,38.0,w,Individual
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,MORTGAGE,110000.0,Source Verified,Paid,debt_consolidation,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,13.0,0.0,7802.0,11.6,17.0,w,Individual
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,MORTGAGE,104433.0,Source Verified,Paid,major_purchase,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,0.0,21929.0,64.5,35.0,w,Individual
5,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,Veterinary Tecnician,RENT,34000.0,Source Verified,Paid,debt_consolidation,GA,10.20,0.0,Oct-1987,690.0,694.0,0.0,5.0,0.0,8822.0,68.4,6.0,w,Individual


In [6]:
# Display the info of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20834 entries, 0 to 22509
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            20834 non-null  float64
 1   funded_amnt          20834 non-null  float64
 2   funded_amnt_inv      20834 non-null  float64
 3   term                 20834 non-null  object 
 4   int_rate             20834 non-null  float64
 5   installment          20834 non-null  float64
 6   grade                20834 non-null  object 
 7   sub_grade            20834 non-null  object 
 8   emp_title            20834 non-null  object 
 9   home_ownership       20834 non-null  object 
 10  annual_inc           20834 non-null  float64
 11  verification_status  20834 non-null  object 
 12  loan_status          20834 non-null  object 
 13  purpose              20834 non-null  object 
 14  addr_state           20834 non-null  object 
 15  dti                  20834 non-null 

In [7]:
# Get a statistical summary of the dataset
df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
count,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,2.083400e+04,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000,20834.000000
mean,15208.433330,15208.433330,15202.132332,12.233967,441.169325,7.998985e+04,19.303227,0.338005,694.998800,698.998896,0.607469,12.218873,0.232361,17644.372564,51.886266,25.596189
std,8757.840042,8757.840042,8752.917757,4.205992,250.437800,6.242066e+04,8.761508,0.886107,31.104621,31.105086,0.889678,5.792621,0.639960,22978.924286,24.112605,12.182928
min,1000.000000,1000.000000,1000.000000,5.320000,30.540000,3.800000e+03,0.000000,0.000000,660.000000,664.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000
25%,8000.000000,8000.000000,8000.000000,9.170000,257.390000,5.000000e+04,12.690000,0.000000,670.000000,674.000000,0.000000,8.000000,0.000000,6366.500000,33.900000,17.000000
50%,14000.000000,14000.000000,14000.000000,11.990000,383.830000,6.900000e+04,18.785000,0.000000,690.000000,694.000000,0.000000,11.000000,0.000000,12060.500000,51.700000,24.000000
75%,20225.000000,20225.000000,20193.750000,14.480000,589.242500,9.600000e+04,25.550000,0.000000,710.000000,714.000000,1.000000,15.000000,0.000000,21475.000000,70.200000,32.000000
max,35000.000000,35000.000000,35000.000000,28.990000,1354.660000,3.964280e+06,136.970000,15.000000,845.000000,850.000000,5.000000,65.000000,23.000000,647310.000000,134.300000,111.000000


#### Numerical Features

In [8]:
# Identify the numerical columns
num_cols = df.select_dtypes(include=np.number).columns.tolist()

# Display the list of numerical columns
num_cols

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc']

In [9]:
# Count the number of numerical columns
num_numerical_columns = len(num_cols)

# Display the count
print(f'Number of numerical columns: {num_numerical_columns}')

Number of numerical columns: 16


#### Categorical Features

In [10]:
# Identify the categorical columns
cat_cols = df.select_dtypes(include='object').columns.tolist()

# Display the list of categorical columns
cat_cols

['term',
 'grade',
 'sub_grade',
 'emp_title',
 'home_ownership',
 'verification_status',
 'loan_status',
 'purpose',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'application_type']

In [11]:
# Count the number of numerical columns
num_categorical_columns = len(cat_cols)

# Display the count
print(f'Number of categorical columns: {num_categorical_columns}')

Number of categorical columns: 12


### Categorical Variables and Dummy Variables

Since we only have 12 categorical variables in the dateset, let's manually dummy variable encode our categorical variables for the following reasons:

1. **Control and Flexibility:** When we manually encode categorical variables, we have full control over the encoding process. We can also customize the column names, and apply specific encoding strategies tailored to our needs.

2. **Transparency:** We can see exactly how the encoding is done, which can be helpful for understanding the data transformation and debugging.

##### 1. Mapping the target variable to 1 and 0

In [12]:
# Map "paid" to 1 and "default" to 0
df['loan_status'] = df['loan_status'].map({'Paid': 1, 'Default': 0})

# Display the first 5 rows of loan_status column
df['loan_status'].head()

0    1
1    1
3    1
4    1
5    1
Name: loan_status, dtype: int64

##### 2. term feature

In [13]:
# Display the number of loans per term value
df['term'].value_counts()

 36 months    14416
 60 months     6418
Name: term, dtype: int64

Let's convert the term feature into either a 36 or 60 integer numeric data type.

In [14]:
# Map '36 months' and '60 months' to 36 and 60 respectively
df['term'] = df['term'].map({' 36 months': 36, ' 60 months': 60})

# Display the first 5 rows of term column
df['term'].head()

0    36
1    36
3    60
4    60
5    36
Name: term, dtype: int64

##### 3. grade and sub_grade feature

We already know grade is part of sub_grade, so let's just drop the grade feature.

In [15]:
# drop the grade column
df = df.drop('grade',axis=1)

Let's convert the subgrade into dummy variables, then concatenate these new columns to the original dataframe. 

In [16]:
# Convert the 'sub_grade' categorical column into dummy variables.
subgrade_dummies = pd.get_dummies(df['sub_grade'], drop_first=True)

# Update the DataFrame 'df' by replacing the 'sub_grade' column with the dummy variables.
df = pd.concat([df.drop('sub_grade', axis=1), subgrade_dummies], axis=1)

# Print the updated column names of the DataFrame after adding the dummy variables.
print(df.columns)


Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'emp_title', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'application_type', 'A2', 'A3',
       'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2',
       'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'],
      dtype='object')


In [17]:
# Print the names of any remaining categorical columns in the DataFrame.
df.select_dtypes(['object']).columns

Index(['emp_title', 'home_ownership', 'verification_status', 'purpose',
       'addr_state', 'earliest_cr_line', 'initial_list_status',
       'application_type'],
      dtype='object')

##### 4. emp_title feature 

In [18]:
# Display the number of unique employment titles
df['emp_title'].nunique()

10863

In [19]:
# Diplay the value counts of for the column emp_title
df['emp_title'].value_counts()

Teacher                                     457
Manager                                     377
Owner                                       200
Supervisor                                  176
RN                                          166
Registered Nurse                            165
Driver                                      135
Sales                                       129
Project Manager                             122
Director                                    106
President                                    96
owner                                        96
General Manager                              91
manager                                      87
Engineer                                     87
Office Manager                               81
driver                                       79
Vice President                               78
teacher                                      76
Attorney                                     74
Accountant                              

Since there are too many unique job titles to convert to a dummy variable feature. Let's remove the emp_title column.

In [20]:
# Drop the emp_title column
df = df.drop('emp_title',axis=1)

##### 5. addr_state feature

In [21]:
# Diplay the unique values of addr_state
df['addr_state'].nunique()

49

Since there are too many unique job titles to convert to a dummy variable feature. Let's remove the addr_state column.

In [22]:
# Drop the emp_title column
df = df.drop('addr_state',axis=1)

##### 6. earliest_cr_line feature

In [23]:
# Display the first 5 rows of earliest_cr_line column
df['earliest_cr_line'].head()

0    Aug-2003
1    Dec-1999
3    Sep-2008
4    Jun-1998
5    Oct-1987
Name: earliest_cr_line, dtype: object

This is a historical time stamp feature. Let's extract the year from this feature, then convert it to a numeric feature. 

In [24]:
# Adds a new column 'earliest_cr_year' to the DataFrame 'df'
df['earliest_cr_year'] = df['earliest_cr_line'].apply(lambda date:int(date[-4:]))

# Drop the earliest_cr_line column
df = df.drop('earliest_cr_line',axis=1)

In [25]:
# Print the names of any remaining categorical columns in the DataFrame.
df.select_dtypes(['object']).columns

Index(['home_ownership', 'verification_status', 'purpose',
       'initial_list_status', 'application_type'],
      dtype='object')

##### 7. home_ownership, verification_status, purpose, initial_list_status, application_type features

In [26]:
# Define the categorical columns
cat_list = ['home_ownership', 'verification_status', 'purpose', 'initial_list_status', 'application_type']

# Get the value counts in each categorical column
for col in cat_list:
    value_counts = df[col].value_counts()
    print(f"Value Counts for {col}:\n{value_counts}\n")

Value Counts for home_ownership:
MORTGAGE    10289
RENT         8260
OWN          2285
Name: home_ownership, dtype: int64

Value Counts for verification_status:
Source Verified    8600
Not Verified       6323
Verified           5911
Name: verification_status, dtype: int64

Value Counts for purpose:
debt_consolidation    11884
credit_card            5193
home_improvement       1169
other                  1127
major_purchase          448
medical                 227
small_business          222
car                     192
moving                  136
vacation                131
house                    91
renewable_energy         14
Name: purpose, dtype: int64

Value Counts for initial_list_status:
w    18141
f     2693
Name: initial_list_status, dtype: int64

Value Counts for application_type:
Individual    20695
Joint App       139
Name: application_type, dtype: int64



In [27]:
# Convert these categorical columns into dummy variables.
dummies = pd.get_dummies(df[['home_ownership', 'verification_status', 'purpose', 'initial_list_status', 'application_type']], drop_first=True)

# Update the DataFrame 'df' by replacing the columns with the dummy variables.
df = pd.concat([df.drop(['home_ownership', 'verification_status', 'purpose', 'initial_list_status', 'application_type'], axis=1), dummies], axis=1)

# Print the updated column names of the DataFrame after adding the dummy variables.
print(df.columns)

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'annual_inc', 'loan_status', 'dti', 'delinq_2yrs',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'A2', 'A3', 'A4',
       'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1',
       'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3',
       'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5', 'earliest_cr_year',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase',
       'purpose_medical', 'purpose_moving', 'purpose_other',
       'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'initial_list_status_w',
       'application_type_Joint App

In [28]:
# Print the names of any remaining categorical columns in the DataFrame.
df.select_dtypes(['object']).columns

Index([], dtype='object')

### Train and Test Split

In [29]:
# Define the features and target
X = df.drop('loan_status',axis=1).values
y = df['loan_status'].values

# Split the data into train and test datasets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Normalizing the Data

Let's normalize the feature data X_train and X_test. Since we don't want data leakge from the test set, we only fit on the X_train data.

In [30]:
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
 
# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Apply the scaler to the test data
X_test = scaler.transform(X_test)

### Apply SMOTE to the Training Set (dealing with imbalanced classes)

In [31]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [32]:
# Print the shapes of the new X_train dataset
print(X_train_sm.shape)

(27396, 69)


In [33]:
# Print the shapes of the new y_train dataset
print(y_train_sm.shape)

(27396,)


In [35]:
# check what SMOTE has done

print('Original class distribution')
display(pd.Series(y_train).value_counts().sort_index())

print('\nResampled class distribution')
display(pd.Series(y_train_sm).value_counts().sort_index())

Original class distribution


0     2969
1    13698
dtype: int64


Resampled class distribution


0    13698
1    13698
dtype: int64

### Convert the Data into DataFrames

In [39]:
# Convert the resampled training data (X_train_resampled and y_train_resampled) into a DataFrame
train_df = pd.DataFrame(np.column_stack((X_train_sm, y_train_sm)), columns=df.drop('loan_status', axis=1).columns.tolist() + ['loan_status'])

# Convert the test data (X_test and y_test) into a DataFrame
test_df = pd.DataFrame(np.column_stack((X_test, y_test)), columns=df.drop('loan_status', axis=1).columns.tolist() + ['loan_status'])

### Train Data

In [40]:
# Display the final train dataset
train_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
0,0.411765,0.411765,0.411765,0.0,0.162653,0.338074,0.015893,0.089071,0.000000,0.108108,0.107527,0.0,0.140625,0.043478,0.013267,0.447506,0.065421,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.676471,0.676471,0.676471,1.0,0.577524,0.447021,0.026730,0.253851,0.066667,0.027027,0.026882,0.0,0.140625,0.000000,0.031700,0.483246,0.168224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.872727,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.411765,0.411765,0.411765,0.0,0.189269,0.341412,0.046797,0.128641,0.000000,0.270270,0.268817,0.6,0.281250,0.000000,0.040715,0.294862,0.261682,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.316176,0.316176,0.316176,0.0,0.281791,0.271637,0.007866,0.218004,0.000000,0.054054,0.053763,0.0,0.062500,0.000000,0.020613,0.588235,0.196262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.690909,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.264706,0.264706,0.264706,0.0,0.224757,0.222903,0.011880,0.087391,0.000000,0.108108,0.107527,0.2,0.046875,0.000000,0.008994,0.446761,0.056075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [41]:
# Display the shape of the training data
train_df.shape

(27396, 70)

In [42]:
# Check for missing values in the dataframe
train_df.isna().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term                                   0
int_rate                               0
installment                            0
annual_inc                             0
dti                                    0
delinq_2yrs                            0
fico_range_low                         0
fico_range_high                        0
inq_last_6mths                         0
open_acc                               0
pub_rec                                0
revol_bal                              0
revol_util                             0
total_acc                              0
A2                                     0
A3                                     0
A4                                     0
A5                                     0
B1                                     0
B2                                     0
B3                                     0
B4              

### Test Data

In [43]:
# Display the final test dataset
test_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,earliest_cr_year,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App,loan_status
0,0.794118,0.794118,0.794118,0.0,0.108576,0.638507,0.277573,0.023582,0.000000,0.243243,0.241935,0.0,0.125000,0.0,0.098225,0.588235,0.149533,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.745455,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.464706,0.464706,0.464706,0.0,0.281791,0.398295,0.013887,0.167044,0.000000,0.162162,0.161290,0.0,0.156250,0.0,0.009498,0.423678,0.196262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.676471,0.676471,0.676471,1.0,0.133925,0.348722,0.056831,0.173031,0.066667,0.135135,0.134409,0.0,0.171875,0.0,0.340650,0.681310,0.261682,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618182,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.676471,0.676471,0.676471,0.0,0.000000,0.522777,0.046797,0.179163,0.000000,0.459459,0.456989,0.0,0.187500,0.0,0.111296,0.453462,0.214953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.581818,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.676471,0.676471,0.675000,1.0,0.249261,0.373018,0.022716,0.138424,0.000000,0.297297,0.295699,0.0,0.125000,0.0,0.009972,0.331348,0.084112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.909091,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [44]:
# Display the shape of the testing data
test_df.shape

(4167, 70)

In [45]:
# Check for missing values in the dataframe
test_df.isna().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term                                   0
int_rate                               0
installment                            0
annual_inc                             0
dti                                    0
delinq_2yrs                            0
fico_range_low                         0
fico_range_high                        0
inq_last_6mths                         0
open_acc                               0
pub_rec                                0
revol_bal                              0
revol_util                             0
total_acc                              0
A2                                     0
A3                                     0
A4                                     0
A5                                     0
B1                                     0
B2                                     0
B3                                     0
B4              

### Save the Datasets

In [46]:
# Save the final train and test datasets to Parquet format
train_df.to_parquet('dataset/train_dataset.parquet', index=False)
test_df.to_parquet('dataset/test_dataset.parquet', index=False)